<a href="https://colab.research.google.com/github/trngbro/AI/blob/main/Progress20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

init

In [11]:
!pip install python-sat==0.1.7.dev12

from pysat.solvers import Glucose3
import itertools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from pysat.solvers import Glucose3
import itertools

def solve_puzzle(matrix):
    n_rows, n_cols = len(matrix), len(matrix[0])
    props = list(itertools.product(range(n_rows), range(n_cols)))
    symbols = {}
    for p in props:
        symbols[p] = 'p{}_{}'.format(p[0], p[1])

    clauses = []
    for i in range(n_rows):
        for j in range(n_cols):
            if isinstance(matrix[i][j], int):
                green_neighbors = []
                for x in range(max(0, i-1), min(n_rows, i+2)):
                    for y in range(max(0, j-1), min(n_cols, j+2)):
                        if isinstance(matrix[x][y], int):
                            continue
                        if (x,y) != (i,j):
                            green_neighbors.append(symbols[(x,y)])
                # if there are no green neighbors, the cell must be red
                if len(green_neighbors) == 0:
                    clauses.append([-symbols[(i,j)]])
                # if there are exactly n green neighbors, the cell must be green
                elif len(green_neighbors) == matrix[i][j]:
                    clauses.append(green_neighbors)
                    clauses.append([symbols[(i,j)]])
                # otherwise, the cell cannot be green
                else:
                    clause = [-symbols[(i,j)]]
                    for g in green_neighbors:
                        clause.append(-g)
                    clauses.append(clause)

    with Glucose3(bootstrap_with=clauses) as solver:
        res = solver.solve()
        if res == True:
            model = solver.get_model()
            result = [['red' for j in range(n_cols)] for i in range(n_rows)]
            for p in props:
                if symbols[p] in model:
                    result[p[0]][p[1]] = 'green'
            return result
        else:
            print('No solution found')
            return None


In [24]:
from pysat.solvers import Glucose3
from itertools import combinations, product

def get_propositions(matrix):
    props = []
    for i in range(len(matrix)):
        for j in range(len(matrix[0])):
            if matrix[i][j] != '':
                props.append((i, j))
    return props

def get_neighbors(prop, matrix):
    neighbors = []
    i, j = prop
    for x in range(max(0, i-1), min(i+2, len(matrix))):
        for y in range(max(0, j-1), min(j+2, len(matrix[0]))):
            if matrix[x][y] != '':
                neighbors.append((x, y))
    return neighbors

def generate_clauses(props, matrix):
    clauses = []
    for p in props:
        neighbors = get_neighbors(p, matrix)
        num_green = matrix[p[0]][p[1]]
        for comb in combinations(neighbors, num_green):
            clause = []
            for n in neighbors:
                if n in comb:
                    clause.append(get_index(n, props) + 1)
                else:
                    clause.append(-(get_index(n, props) + 1))
            clauses.append(clause)
    return clauses

def get_index(prop, props):
    return props.index(prop)

def solve_puzzle(matrix):
    props = get_propositions(matrix)
    clauses = generate_clauses(props, matrix)
    with Glucose3() as solver:
        for clause in clauses:
            solver.add_clause(clause)
        res = solver.solve()
        if res == False:
            return None
        result = [['red' for j in range(len(matrix[0]))] for i in range(len(matrix))]
        for p in props:
            if solver.get_model()[get_index(p, props)] > 0:
                result[p[0]][p[1]] = 'green'
        return result


In [25]:
# Test the function with example input
matrix = [
    [0, '', 1],
    ['', '', ''],
    [2, '', 1],
]
print(solve_puzzle(matrix))

[['red', 'red', 'green'], ['red', 'red', 'red'], ['red', 'red', 'green']]
